# Original Code for scoring private condominium data -- only quantity score.

Ansel Lim

1 November 2021

Given URA dataset (private residential property transactions; https://www.ura.gov.sg/maps/api/#private-residential-property-transactions) as well as other datasets (taxi stands, primary schools, mrt, hawker, carparks, bus stops, chas clinics, sports facilities, community centers), calculate the NUMBER of each of these places within a specified RADIUS (e.g. 1 kilometer) from each property.

In [3]:
# Specify radius in kilometers
RADIUS=1.0

In [4]:
from datetime import datetime
import pandas as pd
from geopy.distance import geodesic
import re
import time

In [5]:
timestamp=datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print(timestamp) # GMT

03/11/2021 12:55:50


In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
%cd /content/gdrive/MyDrive/6242_PROJECT/data

/content/gdrive/MyDrive/6242_PROJECT/data


In [8]:
%ls

amenities.csv              eating-establishments.kml
bus_stops.csv              general-information-of-schools.csv
carparks.csv               listing-of-supermarkets.csv
data-download.png          processed/
data_hawker.csv            secondary_schools.csv
data_hawker.gsheet         secondary_schools.gsheet
data_malls.csv             supermarkets.csv
data_MRT.csv               taxi_stands.csv
data_prischools.csv        ura.csv
data_prischools.gsheet     ura_join_bus_stops.csv
eating_establishments.csv


In [9]:
# somewhere in sentosa cove
lat1=1.242440985
long1=103.8367552
sentosa = (lat1,long1)

# alexandra view condominium near redhill
lat2=1.290030671
long2=103.8171853
redhill = (lat2,long2)

In [10]:
geodesic(sentosa,redhill).km

5.695136317712519

In [35]:
ura = pd.read_csv("./ura.csv")
malls = pd.read_csv("./data_malls.csv")
taxi_stands = pd.read_csv("./taxi_stands.csv")
primary_schools=pd.read_csv("./data_prischools.csv")
mrt = pd.read_csv("./data_MRT.csv")
hawker=pd.read_csv("./data_hawker.csv")
carparks=pd.read_csv("./carparks.csv")
bus_stops = pd.read_csv("./bus_stops.csv")
amenities = pd.read_csv("./amenities.csv")
supermarkets = pd.read_csv("./supermarkets.csv")
secondary_schools=pd.read_csv("./secondary_schools.csv")
eating_establishments=pd.read_csv("./eating_establishments.csv")
parks = pd.read_csv("./parks.csv")

In [12]:
df = ura.copy(deep=True)

In [13]:
taxi_stands["lat"],taxi_stands["long"] = taxi_stands["Latitude"],taxi_stands["Longitude"]
primary_schools[['long','lat']]=primary_schools['coordinates'].str.split(',',1,expand=True)
mrt[['long','lat']]=mrt['Coordinates'].str.split(',',1,expand=True)
hawker[['long','lat']]=hawker['Coordinates'].str.split(',',1,expand=True)
hawker['lat']=hawker['lat'].str.rstrip(",0.0")
carparks['lat'],carparks['long']=carparks['latitude'],carparks['longitude']
bus_stops['lat'],bus_stops['long']=bus_stops['Latitude'],bus_stops['Longitude']

In [14]:
# amenities['coordinates'] = amenities["geometry"].str.replace(r"[a-zA-Z()]",'')

In [15]:
hawker

,ID,Category,Name,Coordinates,rating,user_ratings_total,long,lat
0,1,Hawker Centre,Bedok North Street 4 Blk 85 (85 Fengshan Centre),"103.938732580554,1.33198706861747,0.0",4.3,1064,103.938732580554,1.33198706861747
1,2,Hawker Centre,Redhill Lane Blk 85 (Redhill Food Centre),"103.818338919994,1.28733076703987,0.0",4.2,1942,103.818338919994,1.28733076703987
2,3,Hawker Centre,Sembawang Hills Food Centre (Jalan Leban Food ...,"103.828994204342,1.37238494002145,0.0",3.9,1891,103.828994204342,1.37238494002145
3,4,Hawker Centre,Serangoon Garden Market,"103.866737484646,1.3631571201113,0.0",4.1,2436,103.866737484646,1.3631571201113
4,5,Hawker Centre,Shunfu Road Blk 320 (Shunfu Mart),"103.837031803823,1.35200744198417,0.0",3.9,1574,103.837031803823,1.35200744198417
...,...,...,...,...,...,...,...,...
114,121,Hawker Centre,Jurong West Street 52 Blk 505,"103.71844754363,1.34964224318434,0.0",4.0,3824,103.71844754363,1.34964224318434
115,122,Hawker Centre,Kallang Estate Fresh Market and Food Centre,"103.884130183315,1.30739616009942,0.0",4.1,581,103.884130183315,1.30739616009942
116,123,Hawker Centre,Kampung Admiralty Hawker Centre,"103.800696448529,1.43993283228134,0.0",4.1,4060,103.800696448529,1.43993283228134
117,124,Hawker Centre,Marine Parade Central Blk 84 (84 Marine Parade...,"103.906338526157,1.30228513841822,0.0",4.0,1177,103.906338526157,1.30228513841822


In [16]:
def getLatLong(x):
  coordinates = x['coordinates']
  lat,long=re.findall('[0-9.]+', coordinates)[:2] # get first pair of coordinates (because some geometries are polygons)
  return (long,lat) 

In [17]:
# getLatLong(amenities.loc[0,:])

In [18]:
# amenities[['lat','long']] = amenities.apply(getLatLong,axis=1,result_type='expand')

In [19]:
amenities

,facility_name,facility_type,address,lat,long
0,CHEN FAMILY CLINIC,CHAS Clinic,153 SERANGOON NORTH AVE 1 550153,1.369951,103.872312
1,CHENG CLINIC,CHAS Clinic,447 ANG MO KIO AVE 10 560447,1.367479,103.855967
2,CHEONG FAMILY CLINIC PTE LTD,CHAS Clinic,45 OWEN RD 210045,1.315596,103.850825
3,CHEONG MEDICAL CLINIC,CHAS Clinic,76 JLN JURONG KECHIL 598589,1.340549,103.773330
4,CHERN MEDICAL CLINIC,CHAS Clinic,138 TAMPINES ST 11 521138,1.345505,103.945588
...,...,...,...,...,...
1476,Sports SG Sports Hall/Stadium/Gym Bukit Batok ...,Sports Hall/Stadium/Gym,Bukit Batok West Avenue 5,1.358588,103.753000
1477,Sports SG Stadium Bedok North Street 2,Stadium,Bedok North Street 2,1.326171,103.938076
1478,Sports SG Squash Centre/Tennis Centre Burghley...,Squash Centre/Tennis Centre,Burghley Drive,1.359302,103.860543
1479,Sports SG Swimming Complex/Sports Hall/Stadium...,Swimming Complex/Sports Hall/Stadium/Futsal Co...,Choa Chu Kang Street 53,1.391030,103.748633


In [20]:
amenities.facility_type.value_counts()

CHAS Clinic                                                                      1167
Gym                                                                               160
Community Centre                                                                  120
Swimming Complex                                                                    6
Swimming Complex/Sports Hall/Stadium/Gym                                            4
Stadium                                                                             3
Swimming Complex/Sports Hall/Stadium/Tennis Centre/Gym                              2
Field/Tennis Centre                                                                 2
Sports Hall/Stadium/Gym                                                             2
Swimming Complex/Sports Hall/Gym                                                    2
Swimming Complex/Stadium                                                            2
Swimming Complex/Sports Hall/Gym/Hockey Pitch         

In [21]:
sports_facility_types = list(amenities.facility_type.unique())
sports_facility_types.remove('CHAS Clinic')
sports_facility_types.remove('Community Centre')

In [22]:
chas_clinics = amenities[amenities['facility_type']=='CHAS Clinic'].reset_index()
sports_facilities = amenities[amenities['facility_type'].isin(sports_facility_types)].reset_index()
community_centers = amenities[amenities['facility_type']=='Community Centre'].reset_index()

In [23]:
chas_clinics

,index,facility_name,facility_type,address,lat,long
0,0,CHEN FAMILY CLINIC,CHAS Clinic,153 SERANGOON NORTH AVE 1 550153,1.369951,103.872312
1,1,CHENG CLINIC,CHAS Clinic,447 ANG MO KIO AVE 10 560447,1.367479,103.855967
2,2,CHEONG FAMILY CLINIC PTE LTD,CHAS Clinic,45 OWEN RD 210045,1.315596,103.850825
3,3,CHEONG MEDICAL CLINIC,CHAS Clinic,76 JLN JURONG KECHIL 598589,1.340549,103.773330
4,4,CHERN MEDICAL CLINIC,CHAS Clinic,138 TAMPINES ST 11 521138,1.345505,103.945588
...,...,...,...,...,...,...
1162,1162,ZENITH MEDICAL CLINIC,CHAS Clinic,266C PUNGGOL WAY 823266,1.405154,103.897805
1163,1163,Zam Family Clinic,CHAS Clinic,516 CHOA CHU KANG STREET 51 680516,1.390951,103.743525
1164,1164,banyan clinic @ jw,CHAS Clinic,502 JURONG WEST AVENUE 1 640502,1.350501,103.718335
1165,1165,i.Medical & i.Aesthetics,CHAS Clinic,1 NORTHPOINT DRIVE 768019,1.427903,103.836178


In [24]:
places = [malls,taxi_stands,primary_schools,mrt,hawker,carparks,bus_stops,chas_clinics,sports_facilities,community_centers,supermarkets,secondary_schools,eating_establishments,parks]
places_names = ['malls','taxi_stands','primary_schools','mrt','hawker','carparks','bus_stops','chas_clinics','sports_facilities','community_centers','supermarkets','secondary_schools','eating_establishments','parks']

In [25]:
assert len(places)==len(places_names)

In [26]:
# places=[hawker] #uncomment if testing only on hawker
# places_names=['hawker']

In [27]:
# places=[chas_clinics] #uncomment if testing only on chas clinics
# places_names=['chas_clinics']

In [28]:
# places=[sports_facilities] #uncomment if testing only on sports facilities
# places_names=['sports_facilities']

In [29]:
sports_facilities

,index,facility_name,facility_type,address,lat,long
0,1287,The Pit,Gym,50 Tras Street 78989,1.278114,103.844202
1,1288,The Stable Mixed Martial Arts,Gym,100 Turf Club Road 287992,1.334220,103.795898
2,1289,Bishan ClubFITT Gym,Gym,5 Bishan St. 4 579783,1.355283,103.850922
3,1290,The Yoga Place,Gym,328 King George's Ave 208568,1.309922,103.862091
4,1291,Sky Fitness - Serangoon Gardens,Gym,45 Bughley Drive Blk B 559022,1.360010,103.859571
...,...,...,...,...,...,...
189,1476,Sports SG Sports Hall/Stadium/Gym Bukit Batok ...,Sports Hall/Stadium/Gym,Bukit Batok West Avenue 5,1.358588,103.753000
190,1477,Sports SG Stadium Bedok North Street 2,Stadium,Bedok North Street 2,1.326171,103.938076
191,1478,Sports SG Squash Centre/Tennis Centre Burghley...,Squash Centre/Tennis Centre,Burghley Drive,1.359302,103.860543
192,1479,Sports SG Swimming Complex/Sports Hall/Stadium...,Swimming Complex/Sports Hall/Stadium/Futsal Co...,Choa Chu Kang Street 53,1.391030,103.748633


# Calculate number of places of interest within radius, for each property -- except eating_establishments and parks.

In [30]:
start=time.time()
for i in range(len(places)):
  dataframe = places[i]
  place_name = places_names[i]
  print("Working on feature dataframe {}".format(place_name))
  new_column_name = "num_"+place_name
  df[new_column_name] = 0
  for i in range(df.shape[0]):
    lat1 = df.loc[i,"lat"]
    long1 = df.loc[i,"long"]
    origin = (lat1,long1)
    counter = 0
    for j in range(dataframe.shape[0]):
      lat2 = dataframe.loc[j,"lat"]
      long2 = dataframe.loc[j,"long"]
      dest = (lat2,long2)
      dist = geodesic(origin,dest).km
      if dist<=RADIUS:
        counter+=1
    df.loc[i,new_column_name]=counter
end=time.time()
print("time taken: {}".format(end-start))

Working on feature dataframe malls
Working on feature dataframe taxi_stands
Working on feature dataframe primary_schools
Working on feature dataframe mrt
Working on feature dataframe hawker
Working on feature dataframe carparks
Working on feature dataframe bus_stops
Working on feature dataframe chas_clinics
Working on feature dataframe sports_facilities
Working on feature dataframe community_centers
Working on feature dataframe supermarkets
Working on feature dataframe secondary_schools
Working on feature dataframe eating_establishments


KeyError: ignored

In [ ]:
df.to_csv('./processed/df.csv')

# Run section for eating_establishments and parks, without re-running already completed sections.

In [32]:
eating_establishments

,Unnamed: 0,Name,Description,geometry,lat,lon,business_name
0,0,kml_1,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.857675501543 1.29487905112126 0),1.294879,103.857676,OMU
1,1,kml_2,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.857675501543 1.29487905112126 0),1.294879,103.857676,Unknown business name
2,2,kml_3,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.857675501543 1.29487905112126 0),1.294879,103.857676,Unknown business name
3,3,kml_4,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.858505749339 1.28355872540745 0),1.283559,103.858506,PPUDO
4,4,kml_5,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.858505749339 1.28355872540745 0),1.283559,103.858506,Unknown business name
...,...,...,...,...,...,...,...
34373,34373,kml_34374,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.895103199652 1.35963370805796 0),1.359634,103.895103,Unknown business name
34374,34374,kml_34375,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.89359287972 1.39150507204754 0),1.391505,103.893593,Unknown business name
34375,34375,kml_34376,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.742586015051 1.33402529468935 0),1.334025,103.742586,Unknown business name
34376,34376,kml_34377,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.887857189653 1.35530903339479 0),1.355309,103.887857,Unknown business name


In [33]:
eating_establishments_=eating_establishments.rename(columns={'lon':'long'})
places = [eating_establishments_]
places_names= ['eating_establishments_']
start=time.time()
for i in range(len(places)):
  dataframe = places[i]
  place_name = places_names[i]
  print("Working on feature dataframe {}".format(place_name))
  new_column_name = "num_"+place_name
  df[new_column_name] = 0
  for i in range(df.shape[0]):
    lat1 = df.loc[i,"lat"]
    long1 = df.loc[i,"long"]
    origin = (lat1,long1)
    counter = 0
    for j in range(dataframe.shape[0]):
      lat2 = dataframe.loc[j,"lat"]
      long2 = dataframe.loc[j,"long"]
      dest = (lat2,long2)
      dist = geodesic(origin,dest).km
      if dist<=RADIUS:
        counter+=1
    df.loc[i,new_column_name]=counter
end=time.time()
print("time taken: {}".format(end-start))

Working on feature dataframe eating_establishments_
time taken: 13446.645763158798


In [34]:
df.to_csv('./processed/df.csv')

In [ ]:
eating_establishments_=eating_establishments.rename(columns={'lon':'long'})
places = [eating_establishments_]
places_names= ['eating_establishments_']
start=time.time()
for i in range(len(places)):
  dataframe = places[i]
  place_name = places_names[i]
  print("Working on feature dataframe {}".format(place_name))
  new_column_name = "num_"+place_name
  df[new_column_name] = 0
  for i in range(df.shape[0]):
    lat1 = df.loc[i,"lat"]
    long1 = df.loc[i,"long"]
    origin = (lat1,long1)
    counter = 0
    for j in range(dataframe.shape[0]):
      lat2 = dataframe.loc[j,"lat"]
      long2 = dataframe.loc[j,"long"]
      dest = (lat2,long2)
      dist = geodesic(origin,dest).km
      if dist<=RADIUS:
        counter+=1
    df.loc[i,new_column_name]=counter
end=time.time()
print("time taken: {}".format(end-start))

In [36]:

places = [parks]
places_names= ['parks']
start=time.time()
for i in range(len(places)):
  dataframe = places[i]
  place_name = places_names[i]
  print("Working on feature dataframe {}".format(place_name))
  new_column_name = "num_"+place_name
  df[new_column_name] = 0
  for i in range(df.shape[0]):
    lat1 = df.loc[i,"lat"]
    long1 = df.loc[i,"long"]
    origin = (lat1,long1)
    counter = 0
    for j in range(dataframe.shape[0]):
      lat2 = dataframe.loc[j,"lat"]
      long2 = dataframe.loc[j,"long"]
      dest = (lat2,long2)
      dist = geodesic(origin,dest).km
      if dist<=RADIUS:
        counter+=1
    df.loc[i,new_column_name]=counter
end=time.time()
print("time taken: {}".format(end-start))

Working on feature dataframe parks
time taken: 140.69643354415894


In [37]:
df.to_csv('./processed/df.csv')